In [16]:
from pyspark.sql import SparkSession

In [17]:
spark = SparkSession.builder.appName("Analysing Airline data").getOrCreate()

In [18]:
from pyspark.sql.types import Row
from datetime import datetime

In [58]:
airlinesPath="/home/sudeep/sources/github/apache spark/spark getting started/02/demos/datasets/airlines.csv"
flightsPath="/home/sudeep/sources/github/apache spark/spark getting started/02/demos/datasets/flights.csv"
airportsPath="/home/sudeep/sources/github/apache spark/spark getting started/02/demos/datasets/airports.csv"

In [20]:
airlines = spark.read.format('csv').option('header', 'true').load(airlinesPath)

In [21]:
airlines.createOrReplaceTempView("airlines")

In [22]:
airlines =  spark.sql('select * from airlines')

In [23]:
airlines.columns

['Code', 'Description']

In [24]:
airlines.show(5)

+-----+--------------------+
| Code|         Description|
+-----+--------------------+
|19031|Mackey Internatio...|
|19032|Munz Northern Air...|
|19033|Cochise Airlines ...|
|19034|Golden Gate Airli...|
|19035|  Aeromech Inc.: RZZ|
+-----+--------------------+
only showing top 5 rows



In [25]:
flights = spark.read.format('csv').option('header', 'true').load(flightsPath)

In [26]:
flights.createOrReplaceTempView("flights")

In [27]:
flights.columns

['date',
 'airlines',
 'flight_number',
 'origin',
 'destination',
 'departure',
 'departure_delay',
 'arrival',
 'arrival_delay',
 'air_time',
 'distance']

In [28]:
flights.show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            1|   JFK|        LAX|     0854|          -6.00|   1217|         2.00|  355.00| 2475.00|
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
|2014-04-01|   19805|            3|   JFK|        LAX|     1224|          -6.00|   1614|        39.00|  371.00| 2475.00|
|2014-04-01|   19805|            4|   LAX|        JFK|     1240|          25.00|   2028|       -27.00|  264.00| 2475.00|
|2014-04-01|   19805|            5|   DFW|        HNL|     1300|          -5.00|   1650|        15.00|  510.00| 3784.00|
+----------+--------+-----------

In [29]:
flights.count(), airlines.count()

(476881, 1579)

In [30]:
flights_count = spark.sql("select COUNT(*) from flights")

In [31]:
airlines_count = spark.sql("select COUNT(*) from airlines")

In [33]:
flights_count.show()

+--------+
|count(1)|
+--------+
|  476881|
+--------+



In [34]:
airlines_count.show()

+--------+
|count(1)|
+--------+
|    1579|
+--------+



In [38]:
flights_count.collect()[0][0]

476881

In [40]:
total_distance_df = spark.sql("SELECT distance FROM flights").agg({"distance":"sum"}).withColumnRenamed("sum(distance)", "total_distance")

In [41]:
total_distance_df.show()

+--------------+
|total_distance|
+--------------+
|  3.79052917E8|
+--------------+



In [55]:
all_delays_2014 =  spark.sql("SELECT * FROM flights where departure_delay> 0 and year(date) = 2014")
all_delays =  spark.sql("SELECT * FROM flights where departure_delay> 0 ")

In [56]:
all_delays_2014.show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
|2014-04-01|   19805|            4|   LAX|        JFK|     1240|          25.00|   2028|       -27.00|  264.00| 2475.00|
|2014-04-01|   19805|            6|   OGG|        DFW|     1901|         126.00|   0640|        95.00|  385.00| 3711.00|
|2014-04-01|   19805|            7|   DFW|        OGG|     1410|         125.00|   1743|       138.00|  497.00| 3711.00|
|2014-04-01|   19805|            8|   HNL|        DFW|     1659|           4.00|   0458|       -22.00|  398.00| 3784.00|
+----------+--------+-----------

In [57]:
all_delays_2014.orderBy(all_delays_2014.departure_delay.desc()).show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-27|   20366|         5246|   ATL|        CHO|     1219|          99.00|   1340|        87.00|   64.00|  457.00|
|2014-04-27|   19393|         2948|   DEN|        GRR|     1349|          99.00|   1807|        92.00|  127.00| 1015.00|
|2014-04-27|   20366|         5365|   DTW|        TUL|     1720|          99.00|   1831|        82.00|  117.00|  789.00|
|2014-04-26|   19977|          616|   ORD|        DCA|     1542|          99.00|   1828|        92.00|   71.00|  612.00|
|2014-04-27|   20366|         6030|   EWR|        RDU|     1858|          99.00|   2108|       132.00|   64.00|  416.00|
+----------+--------+-----------

In [59]:
all_delays_2014.createOrReplaceTempView('all_delays')
delay_count = spark.sql("SELECT COUNT(departure_delay) from all_delays")

In [60]:
delay_count.show()

+----------------------+
|count(departure_delay)|
+----------------------+
|                179015|
+----------------------+

